In [1]:
'''
    Aluna: Meriely Eline Lima Gaia
    Matrícula: 201633840047

Para realização deste trabalho foi utilizado o ambiente computacional Jupyter Notebook e as seguintes configurações
    - Ubuntu 20.04.1 LTS   
    - Python 3.7
        numpy 1.19.2
        pandas 1.1.3
        sklearn 0.24.1
        plotly 4.14.3
Foram utilizados os seguintes métodos preditivos:
    1. Árvore de Decisão 
    2. Regressão Linear

    Referências: 
        https://www.youtube.com/watch?v=hOLSGMEEwlI&ab_channel=ComputerScience
        https://minerandodados.com.br/como-criar-dashboards-em-python/
        https://minerandodados.com.br/analisando-dados-da-bolsa-de-valores-com-python/
        https://ichi.pro/pt/negociacao-algoritmica-em-python-medias-moveis-simples-8012299612427
        
'''
import warnings
warnings.filterwarnings('ignore') 

In [2]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.graph_objs as go
import plotly
import plotly.offline as py
from plotly.offline import plot, iplot
plotly.offline.init_notebook_mode(connected=True)
import heapq


In [3]:
# armazenar os dados em um dataframe
df1 = pd.read_csv('BTC-USD.csv')
df1.head() # mostra os pprimeiros 5 valores

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-11-07,6468.500000,6552.160156,6468.310059,6530.140137,6530.140137,4941260000
1,2018-11-08,6522.270020,6536.919922,6438.529785,6453.720215,6453.720215,4665260000
2,2018-11-09,6442.600098,6456.459961,6373.370117,6385.620117,6385.620117,4346820000
3,2018-11-10,6386.129883,6437.279785,6385.310059,6409.220215,6409.220215,3705320000
4,2018-11-11,6413.629883,6423.250000,6350.169922,6411.270020,6411.270020,3939060000


In [4]:
'''
Gráfico de Candlestick: Cada vela contém os valores de abertura, alta, baixa e fechamento.
Quando o valor de fechamento da ação é menor que o valor de abertura temos uma vela vermelha, 
se o valor fechar acima do preço de abertura a vela é verde. 
'''
# Visualizar comportamento através do gráfico de candlestick
fig = go.Figure(data=[go.Candlestick(x=df1['Date'],
                open=df1['Open'],
                high=df1['High'],
                low=df1['Low'],
                close=df1['Close'])])

#fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(
    title = 'Preço dos bitcoins durante os anos de 2018 à 2020',
    yaxis_title = 'Preço USD ($)',
    xaxis_title = 'Data')
fig.show()

In [5]:
# Mudar nome das colunas e fechar dados para jan 2019 à outubro 2020
df1.columns = ['data', 'abertura', 'max', 'min', 'fechamento', 'ajus_fechamento', 'volume']
df1 = df1[0:725] 
df1.tail().round(2) # mostra os 5 ultimos valores

,data,abertura,max,min,fechamento,ajus_fechamento,volume
720,2020-10-27,13075.24,13759.67,13060.84,13654.22,13654.22,33749878155
721,2020-10-28,13654.21,13837.70,12932.25,13271.29,13271.29,35867318894
722,2020-10-29,13271.30,13612.05,12980.06,13437.88,13437.88,56499499598
723,2020-10-30,13437.87,13651.52,13136.20,13546.52,13546.52,30581485200
724,2020-10-31,13546.53,14028.21,13457.53,13781.00,13781.00,30306464718


In [6]:
#obter o número de dias de negociação
df1.shape[0]

725

In [7]:
df1.describe()

,abertura,max,min,fechamento,ajus_fechamento,volume
count,725.000000,725.000000,725.000000,725.000000,725.000000,7.250000e+02
mean,8025.585766,8191.837972,7860.306388,8035.995001,8035.995001,2.176276e+10
std,2627.744526,2681.415560,2574.146714,2636.210920,2636.210920,1.213785e+10
min,3236.274658,3275.377930,3191.303467,3236.761719,3236.761719,3.551764e+09
25%,5831.374512,6183.039063,5708.035156,5830.254883,5830.254883,1.373869e+10
50%,8599.758789,8745.590820,8395.107422,8601.795898,8601.795898,1.976013e+10
75%,10024.115234,10191.675781,9729.334961,10051.704102,10051.704102,2.870507e+10
max,13654.214844,14028.213867,13457.530273,13780.995117,13780.995117,7.415677e+10


In [8]:
# Pegar apenas os preços de fechamento
df = df1[['fechamento']] # df é apenas a coluna fechamento
df.tail(4)

,fechamento
721,13271.285156
722,13437.882813
723,13546.522461
724,13780.995117


In [9]:
# visualização do dados de preço de fechamento
fig_fechamento = {
    'x': df1.data,
    'y': df.fechamento,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'blue'
    }
}
# informar todos os dados e gráficos em uma lista
data = [fig_fechamento]
 
# configurar o layout do gráfico
layout = go.Layout(title='Preço de fechamento dos bitcoins durante os anos de 2018 à 2020 ',

                   # Definindo exibicao dos eixos x e y
                   yaxis={'title':'Valor do Bitcoin', 
                          'tickformat':'.', 
                          'tickprefix':'$ '},
                   xaxis={'title': 'Dias',
                          })
 
# instanciar objeto Figure e plotar o gráfico
fig = go.Figure(data=data, layout=layout)
fig.show()

In [10]:
# Criar uma variável para prever 'x' dias no futuro
dias_futuro = 31

# Criar uma nova coluna (alvo) deslocada 'x' unidades/dias para cima
df['predicao'] = df[['fechamento']].shift(-dias_futuro)# df é a coluna fechamento + a coluna predição
df.tail(4) 

,fechamento,predicao
721,13271.285156,NaN
722,13437.882813,NaN
723,13546.522461,NaN
724,13780.995117,NaN


In [11]:
#criar a feature dos dados (X) e converta-la em uma matriz numpy + remover as últimas 'x' linhas/dias

X = np.array(df.drop(['predicao'], 1))[:-dias_futuro]
print("Tamanho depois da remoção do dias", X.shape)
X.round(2)

Tamanho depois da remoção do dias (694, 1)


array([[ 6530.14],
       [ 6453.72],
       [ 6385.62],
       [ 6409.22],
       [ 6411.27],
       [ 6371.27],
       [ 6359.49],
       [ 5738.35],
       [ 5648.03],
       [ 5575.55],
       [ 5554.33],
       [ 5623.54],
       [ 4871.49],
       [ 4451.87],
       [ 4602.17],
       [ 4365.94],
       [ 4347.11],
       [ 3880.76],
       [ 4009.97],
       [ 3779.13],
       [ 3820.72],
       [ 4257.42],
       [ 4278.85],
       [ 4017.27],
       [ 4214.67],
       [ 4139.88],
       [ 3894.13],
       [ 3956.89],
       [ 3753.99],
       [ 3521.1 ],
       [ 3419.94],
       [ 3476.11],
       [ 3614.23],
       [ 3502.66],
       [ 3424.59],
       [ 3486.95],
       [ 3313.68],
       [ 3242.48],
       [ 3236.76],
       [ 3252.84],
       [ 3545.86],
       [ 3696.06],
       [ 3745.95],
       [ 4134.44],
       [ 3896.54],
       [ 4014.18],
       [ 3998.98],
       [ 4078.6 ],
       [ 3815.49],
       [ 3857.3 ],
       [ 3654.83],
       [ 3923.92],
       [ 382

In [12]:
#criar o conjunto de dados de destino (y) e converte-lo em uma matriz numpy 
# obter todos os valores do alvo (y), exceto as últimas 'x' linhas/dias
y = np.array(df['predicao'])[:-dias_futuro]
y.round(2)

array([ 3476.11,  3614.23,  3502.66,  3424.59,  3486.95,  3313.68,
        3242.48,  3236.76,  3252.84,  3545.86,  3696.06,  3745.95,
        4134.44,  3896.54,  4014.18,  3998.98,  4078.6 ,  3815.49,
        3857.3 ,  3654.83,  3923.92,  3820.41,  3865.95,  3742.7 ,
        3843.52,  3943.41,  3836.74,  3857.72,  3845.19,  4076.63,
        4025.25,  4030.85,  4035.3 ,  3678.92,  3687.37,  3661.3 ,
        3552.95,  3706.05,  3630.68,  3655.01,  3678.56,  3657.84,
        3728.57,  3601.01,  3576.03,  3604.58,  3585.12,  3600.87,
        3599.77,  3602.46,  3583.97,  3470.45,  3448.12,  3486.18,
        3457.79,  3487.95,  3521.06,  3464.01,  3459.15,  3466.36,
        3413.77,  3399.47,  3666.78,  3671.2 ,  3690.19,  3648.43,
        3653.53,  3632.07,  3616.88,  3620.81,  3629.79,  3673.84,
        3915.71,  3947.09,  3999.82,  3954.12,  4005.53,  4142.53,
        3810.43,  3882.7 ,  3854.36,  3851.05,  3854.79,  3859.58,
        3864.42,  3847.18,  3761.56,  3896.38,  3903.94,  3911

In [13]:
# Dividir (split) os dados em 75% para treinamento e 25% para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [14]:
X_train.shape

(520, 1)

In [15]:
X_test.shape

(174, 1)

In [16]:
# ---------- Criar os modelos ---------- # 

#Criar o modelo de regressão da árvore de decisão
arvore = DecisionTreeRegressor().fit(X_train, y_train)

#Criar o modelo de regressão linear

linear = LinearRegression().fit(X_train, y_train)

In [17]:
#testando o modelo a fim retornar a acurácia da previsão
print("Acurácia do modelo de arvore de decisão (R quadrado) = {}".format(arvore.score(X_test, y_test)))
print("Acurácia do modelo de regressão linerar (R quadrado) = {}".format(linear.score(X_test, y_test)))


Acurácia do modelo de arvore de decisão (R quadrado) = 0.5640898238864767
Acurácia do modelo de regressão linerar (R quadrado) = 0.6431376120881702


In [18]:
# obter as últimas 'x' linhas do conjunto de dados
x_futuro = df.drop(['predicao'], 1)[: -dias_futuro]
x_futuro = x_futuro.tail(dias_futuro)
x_futuro.head()

,fechamento
663,11680.820313
664,11970.478516
665,11414.034180
666,10245.296875
667,10511.813477


In [19]:
# converte-lo em uma matriz numpy 
x_futuro = np.array(x_futuro)
print("Tamanho: ", x_futuro.shape)
x_futuro.round(2)

Tamanho:  (31, 1)


array([[11680.82],
       [11970.48],
       [11414.03],
       [10245.3 ],
       [10511.81],
       [10169.57],
       [10280.35],
       [10369.56],
       [10131.52],
       [10242.35],
       [10363.14],
       [10400.92],
       [10442.17],
       [10323.76],
       [10680.84],
       [10796.95],
       [10974.91],
       [10948.99],
       [10944.59],
       [11094.35],
       [10938.27],
       [10462.26],
       [10538.46],
       [10225.86],
       [10745.55],
       [10702.29],
       [10754.44],
       [10774.43],
       [10721.33],
       [10848.83],
       [10787.62]])

In [26]:
# Mostrar o modelo de regressão linear 
predicao_linear = linear.predict(x_futuro)
print("\nVetor com os valores do modelo de regressão linear: \n",predicao_linear.round(2))

# Mostrar o modelo de arvores de decisão
predicao_arvore = arvore.predict(x_futuro)
print("\nVetor com valores do modelo de arvores de decisão: \n",predicao_arvore.round(2))



Vetor com os valores do modelo de regressão linear: 
 [11282.16 11515.82 11066.95 10124.17 10339.16 10063.08 10152.45 10224.41
 10032.39 10121.79 10219.23 10249.7  10282.98 10187.46 10475.51 10569.17
 10712.72 10691.82 10688.26 10809.07 10683.17 10299.19 10360.65 10108.49
 10527.71 10492.81 10534.88 10551.   10508.17 10611.02 10561.64]

Vetor com valores do modelo de arvores de decisão: 
 [10623.33  9607.42 10565.49 11296.36 10804.   10621.66 10679.14 10923.63
 11079.47 11296.36  8374.69 11555.36 11555.36 11429.51 11495.35 11322.12
 11358.1  11483.36 11742.04 11916.33 11742.04 10804.   12931.54 13108.06
 13031.17  9900.77 13654.22 13271.29  9900.77 13546.52 11322.12]


In [31]:
num_max_1 = np.argpartition(predicao_arvore, -5)[-5:]
print("1. ARVORE DE DECISÃO - OUTUBRO 2020\n")
print(" ----- MÁXIMO -----")
print("Nos dias {}".format(num_max_1), "o preço do bitcoin estará mais valorizado.")
for i in range(4):
    index = num_max_1[i]
    valor = predicao_arvore[index]
  
    print("No dia",num_max_1[i], "o valor será de: $", valor.round(2))
    
num_min_1 = np.argpartition(predicao_arvore, 5)
print("\n ---- MÍNIMO -----")
print("Nos dias ",num_min_1[0:4], "o preço do bitcoin estará mais desvalorizado.")
for i in range(4):
    index = num_min_1[i]
    valor = predicao_arvore[index]
  
    print("No dia",num_min_1[i], "o valor será de: $", valor.round(2))


1. ARVORE DE DECISÃO - OUTUBRO 2020

 ----- MÁXIMO -----
Nos dias [24 23 27 29 26] o preço do bitcoin estará mais valorizado.
No dia 24 o valor será de: $ 13031.17
No dia 23 o valor será de: $ 13108.06
No dia 27 o valor será de: $ 13271.29
No dia 29 o valor será de: $ 13546.52

 ---- MÍNIMO -----
Nos dias  [ 1 10 28 25] o preço do bitcoin estará mais desvalorizado.
No dia 1 o valor será de: $ 9607.42
No dia 10 o valor será de: $ 8374.69
No dia 28 o valor será de: $ 9900.77
No dia 25 o valor será de: $ 9900.77


In [32]:
num_max_2 = np.argpartition(predicao_linear, -5)[-5:]
print("2. REGRESSÃO LINEAR - OUTUBRO 2020\n")
print(" ----- MÁXIMO -----")
print("Nos dias",num_max_2, "o preço do bitcoin estará mais valorizado.")
for i in range(4):
    index = num_max_2[i]
    valor = predicao_linear[index]
  
    print("No dia",num_max_2[i], "o valor será de: $", valor.round(2))
 
num_min_2 = np.argpartition(predicao_linear, 5)
print("\n ---- MÍNIMO -----")
print("Nos dias ",num_min_2[0:4], "o preço do bitcoin estará mais desvalorizado.")
for i in range(4):
    index = num_min_2[i]
    valor = predicao_linear[index]
  
    print("No dia",num_min_2[i], "o valor será de: $", valor.round(2))


2. REGRESSÃO LINEAR - OUTUBRO 2020

 ----- MÁXIMO -----
Nos dias [16  1  2 19  0] o preço do bitcoin estará mais valorizado.
No dia 16 o valor será de: $ 10712.72
No dia 1 o valor será de: $ 11515.82
No dia 2 o valor será de: $ 11066.95
No dia 19 o valor será de: $ 10809.07

 ---- MÍNIMO -----
Nos dias  [ 5  8 23  9] o preço do bitcoin estará mais desvalorizado.
No dia 5 o valor será de: $ 10063.08
No dia 8 o valor será de: $ 10032.39
No dia 23 o valor será de: $ 10108.49
No dia 9 o valor será de: $ 10121.79


In [23]:
# criar variável para mostrar apenas os valores referentes ao mes de outubro de 2020
ultimas_datas = df1.data[694:725]
ultimas_datas.to_frame().head()

,data
694,2020-10-01
695,2020-10-02
696,2020-10-03
697,2020-10-04
698,2020-10-05


In [24]:
# Visualizar os dados obtidos de forma iterativa a partir do modelo de Regressão Linear

validacao_linear = df[X.shape[0]:]
validacao_linear['predicoes_linear'] = predicao_linear
original = {
    'x': ultimas_datas,
    'y': validacao_linear.fechamento,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'blue'
    },
    'name': 'Original'
}
predicao = {
    'x': ultimas_datas,
    'y': validacao_linear.predicoes_linear,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'green'
    },
    'name': 'Predição'
}

# informar todos os dados e gráficos em uma lista
data = [original, predicao]
 
# configurar o layout do gráfico
layout = go.Layout(title='Modelo de Regressão Linear',

                   # Definindo exibicao dos eixos x e y
                   yaxis={'title':'Valor de Fechamento USD($)', 
                          'tickformat':'.', 
                          'tickprefix':'$ '},
                   xaxis={'title': 'Dias',
                          })
 
# instanciar objeto Figure e plotar o gráfico
fig = go.Figure(data=data, layout=layout)
fig.show()

In [25]:
## Visualizar os dados obtidos de forma iterativa a partir do modelo de arvore de decisão

validacao_arvore = df[X.shape[0]:]
validacao_arvore['predicoes_arvore'] = predicao_arvore

original = {
    'x': ultimas_datas,
    'y': validacao_arvore.fechamento,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'blue'
    },
    'name': 'Original'
}
predicao = {
    'x': ultimas_datas,
    'y': validacao_arvore.predicoes_arvore,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'green'
    },
    'name': 'Predição'
}

# informar todos os dados e gráficos em uma lista
data = [original, predicao]
 
# configurar o layout do gráfico
layout = go.Layout(title='Modelo de Arvore de Decisão',

                   # Definindo exibicao dos eixos x e y
                   yaxis={'title':'Valor de Fechamento USD($)', 
                          'tickformat':'.', 
                          'tickprefix':'$ '},
                   xaxis={'title': 'Dias',
                          })
 
# instanciar objeto Figure e plotar o gráfico
fig = go.Figure(data=data, layout=layout)
fig.show()